In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd /content/gdrive/My Drive/Colab Notebooks/Club Mahindra DataOlympics

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
from datetime import date

def days(chkin, chkout):

  ndays = []

  for i in range(len(chkin)):

    date_format = "%d/%m/%Y"
    d1, m1, y1 = int(chkin[i].split('/')[0]), int(chkin[i].split('/')[1]), int(chkin[i].split('/')[2])
    d2, m2, y2 = int(chkout[i].split('/')[0]), int(chkout[i].split('/')[1]), int(chkout[i].split('/')[2])

    date1 = date(y1, m1, d1)
    date2 = date(y2, m2, d2)

    difference = date2 - date1
    
    temp = [int(t) for t in str(difference).split() if t.isdigit()]
    if len(temp) == 0:
      ndays.append(0)
    else:
      ndays.append(temp[0])
  return ndays

In [ ]:
train_advance = days(train['booking_date'], train['checkin_date'])
test_advance = days(test['booking_date'], test['checkin_date'])

In [ ]:
train_ndays = days(train['checkin_date'], train['checkout_date'])
test_ndays = days(test['checkin_date'], test['checkout_date'])

In [ ]:
train.insert(2, "no_of_days_stay", train_ndays)
test.insert(2, "no_of_days_stay", test_ndays)

In [ ]:
train.insert(2, "no_of_days_advance", train_ndays)
test.insert(2, "no_of_days_advance", test_ndays)

In [ ]:
train = train.drop(['reservation_id', 'booking_date', 'checkin_date', 'checkout_date', 'memberid'], axis = 1)
test = test.drop(['reservation_id', 'booking_date', 'checkin_date', 'checkout_date', 'memberid'], axis = 1)

In [ ]:
train['member_age_buckets'].value_counts()

In [ ]:
test['member_age_buckets'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['member_age_buckets'] = le.fit_transform(train['member_age_buckets']) 
test['member_age_buckets'] = le.fit_transform(test['member_age_buckets'])

In [ ]:
train['member_age_buckets'].value_counts()

In [ ]:
test['member_age_buckets'].value_counts()

In [ ]:
train['cluster_code'].value_counts()

In [ ]:
test['cluster_code'].value_counts()

In [ ]:
train['cluster_code'] = le.fit_transform(train['cluster_code']) 
test['cluster_code'] = le.fit_transform(test['cluster_code'])

In [ ]:
train['cluster_code'].value_counts()

In [ ]:
test['cluster_code'].value_counts()

In [ ]:
train['reservationstatusid_code'].value_counts()

In [ ]:
test['reservationstatusid_code'].value_counts()

In [ ]:
train['reservationstatusid_code'] = le.fit_transform(train['reservationstatusid_code'])
test['reservationstatusid_code'] = le.fit_transform(test['reservationstatusid_code'])

In [ ]:
train['reservationstatusid_code'].value_counts()

In [ ]:
test['reservationstatusid_code'].value_counts()

In [ ]:
train['resort_id'].value_counts()

In [ ]:
test['resort_id'].value_counts()

In [ ]:
train['resort_id']=le.fit_transform(train['resort_id'])
test['resort_id']=le.fit_transform(test['resort_id'])

In [ ]:
train['resort_id'].value_counts()

In [ ]:
test['resort_id'].value_counts()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.isnull().sum()             #finding the missing values in train

In [ ]:
test.isnull().sum()             #finding the missing values in test

In [ ]:
train['season_holidayed_code'].fillna((train['season_holidayed_code'].mean()), inplace=True)
test['season_holidayed_code'].fillna((test['season_holidayed_code'].mean()), inplace=True)

In [ ]:
train['state_code_residence'].fillna((train['state_code_residence'].mean()), inplace=True)
test['state_code_residence'].fillna((test['state_code_residence'].mean()), inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
!pip install catboost

In [ ]:
Y = train.iloc[:, -1]
Y.shape

In [ ]:
X = train.drop(['amount_spent_per_room_night_scaled'], axis = 1)
X.shape

In [ ]:
X_test = test

In [ ]:
X_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_valid = scalar.transform(X_valid)
X_test = scalar.transform(X_test)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

model=CatBoostRegressor(iterations=15000, depth=5, learning_rate=0.01, loss_function='RMSE')

In [ ]:
model.fit(X_train, Y_train,eval_set=(X_valid, Y_valid),plot=True)

In [ ]:
Y_test = model.predict(X_test)

In [ ]:
final_submit = pd.read_csv('sample_submission.csv')
final_submit = final_submit.drop(['amount_spent_per_room_night_scaled'], axis = 1)

In [ ]:
final_submit.insert(1, 'amount_spent_per_room_night_scaled', Y_test)

In [ ]:
final_submit.to_csv('final.csv', index = False)

In [ ]:
from google.colab import files
files.download('final.csv')